# Age and Gender Debiasing using UTKFace Dataset with W&B Grid Search

This notebook implements a debiased age prediction model using adversarial training with W&B hyperparameter sweep using grid search.


## 📚 Import Required Libraries


In [2]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout, Layer
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import wandb
try:
    from wandb.keras import WandbCallback
except ImportError:
    from wandb.integration.keras import WandbCallback
from tqdm import tqdm


## 🖼️ Load and Preprocess Images


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jangedoo/utkface-new")
print("Path to dataset files:", path)


Using Colab cache for faster access to the 'utkface-new' dataset.
Path to dataset files: /kaggle/input/utkface-new


In [4]:
# Dataset path
data_dir = os.path.join(path, "UTKFace")

# Lists to store data
images, ages, genders, races = [], [], [], []

for img_name in tqdm(os.listdir(data_dir)):
    try:
        # UTKFace filenames: [age]_[gender]_[race]_*.jpg
        age, gender, race = img_name.split('_')[:3]
        age, gender, race = int(age), int(gender), int(race)

        img_path = os.path.join(data_dir, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (96, 96))
        images.append(img)
        ages.append(age)
        genders.append(gender)
        races.append(race)
    except:
        continue

# Convert to numpy arrays
images = np.array(images)
ages = np.array(ages)
genders = np.array(genders)
races = np.array(races)

print("Total images loaded:", len(images))
print("Example labels:", ages[0], genders[0], races[0])


100%|██████████| 23708/23708 [00:25<00:00, 918.16it/s]


Total images loaded: 23705
Example labels: 26 0 2


In [5]:
# -----------------------
# Gradient Reversal Layer
# -----------------------
class GradientReversalLayer(Layer):
    def __init__(self, alpha=1.0, **kwargs):
        super(GradientReversalLayer, self).__init__(**kwargs)
        self.alpha = alpha

    def call(self, x):
        @tf.custom_gradient
        def _flip_grad(x):
            def grad(dy):
                return -self.alpha * dy
            return x, grad
        return _flip_grad(x)


In [6]:
# -----------------------
# Helper function: Calculate MAE for each group
# -----------------------
def group_mae(true_labels, pred_labels, group_labels, group_names):
    """
    Calculate MAE for each group.
    """
    result = {}
    true_labels = np.array(true_labels).flatten()
    pred_labels = np.array(pred_labels).flatten()
    group_labels = np.array(group_labels).flatten()

    for group_id in np.unique(group_labels):
        mask = group_labels == group_id
        if np.sum(mask) > 0:
            mae = mean_absolute_error(true_labels[mask], pred_labels[mask])
            group_name = group_names.get(int(group_id), f"Group_{group_id}")
            result[group_name] = float(mae)

    return result


In [7]:
# -----------------------
# Prepare data (train/test split)
# -----------------------
X_train, X_test, age_train, age_test, gen_train, gen_test, race_train, race_test = train_test_split(
    images, ages, genders, races, test_size=0.2, random_state=42, shuffle=True
)

# Reshape age labels
age_train = np.array(age_train).reshape(-1, 1).astype('float32')
age_test = np.array(age_test).reshape(-1, 1).astype('float32')

# Process gender labels
gen_train_arr = np.array(gen_train).astype(int)
gen_test_arr = np.array(gen_test).astype(int)
gen_train_cat = to_categorical(gen_train_arr, num_classes=2).astype('float32')
gen_test_cat = to_categorical(gen_test_arr, num_classes=2).astype('float32')
gen_train_labels = gen_train_arr
gen_test_labels = gen_test_arr

# Process race labels
race_train_arr = np.array(race_train).flatten().astype(int)
race_test_arr = np.array(race_test).flatten().astype(int)
race_train_cat = to_categorical(race_train_arr, num_classes=5).astype('float32')
race_test_cat = to_categorical(race_test_arr, num_classes=5).astype('float32')


In [8]:
# -----------------------
# Preprocess images (resize + MobileNet preprocessing)
# -----------------------
def preprocess_images_for_mobilenet(X, target_size=(96, 96)):
    X_tf = tf.convert_to_tensor(X, dtype=tf.float32)
    X_resized = tf.image.resize(X_tf, target_size)
    X_pre = preprocess_input(X_resized)
    return X_pre.numpy()

print("Preprocessing training images...")
X_train_pre = preprocess_images_for_mobilenet(X_train, target_size=(96, 96))

print("Preprocessing test images...")
X_test_pre = preprocess_images_for_mobilenet(X_test, target_size=(96, 96))

print("Prepared image shapes:", X_train_pre.shape, X_test_pre.shape)
print("Label shapes:", age_train.shape, gen_train_cat.shape, race_train_cat.shape)


Preprocessing training images...
Preprocessing test images...
Prepared image shapes: (18964, 96, 96, 3) (4741, 96, 96, 3)
Label shapes: (18964, 1) (18964, 2) (18964, 5)


In [9]:
# -----------------------
# TRAINING FUNCTION FOR W&B SWEEP (STABLE VERSION)
# -----------------------

def train_with_sweep():

    # Initialize W&B run (DO NOT specify project inside sweep)
    run = wandb.init()
    config = wandb.config

    # Hyperparameters
    batch_size = config.batch_size
    learning_rate = config.learning_rate
    dropout_rate = config.dropout_rate
    adv_loss_weight = config.adv_loss_weight
    n_unfreeze = config.n_unfreeze
    reduce_lr_factor = config.reduce_lr_factor
    reduce_lr_patience = config.reduce_lr_patience
    early_stop_patience = config.early_stop_patience
    weight_decay = config.weight_decay

    total_epochs = 50
    grl_alpha = 0.2
    phase1_fraction = 0.7
    phase1_epochs = int(total_epochs * phase1_fraction)
    phase2_epochs = total_epochs - phase1_epochs

    checkpoint_path = f"best_model_{run.id}.keras"

    print("\n" + "="*60)
    print("Training with config:")
    print(dict(config))
    print("="*60)

    # -----------------------
    # BUILD MODEL
    # -----------------------

    input_layer = Input(shape=(96, 96, 3))

    mobilenet_base = MobileNetV2(
        input_shape=(96, 96, 3),
        include_top=False,
        weights='imagenet'
    )
    mobilenet_base.trainable = False

    x = mobilenet_base(input_layer)
    x = GlobalAveragePooling2D()(x)

    concept = Dense(128, activation='relu', name='concept_layer')(x)
    shared = Dropout(dropout_rate)(concept)

    age_output = Dense(1, name='age_output')(shared)

    grl_gender = GradientReversalLayer(alpha=grl_alpha)(concept)
    gender_output = Dense(2, activation='softmax', name='gender_output')(grl_gender)

    grl_race = GradientReversalLayer(alpha=grl_alpha)(concept)
    race_output = Dense(5, activation='softmax', name='race_output')(grl_race)

    dif_model_full = Model(
        inputs=input_layer,
        outputs=[age_output, gender_output, race_output]
    )

    dif_model_age_only = Model(
        inputs=input_layer,
        outputs=age_output
    )

    # -----------------------
    # CALLBACKS (NO W&B CALLBACK)
    # -----------------------

    reduce_lr = ReduceLROnPlateau(
        monitor='val_loss',
        factor=reduce_lr_factor,
        patience=reduce_lr_patience,
        verbose=1,
        min_lr=1e-6
    )

    checkpoint = ModelCheckpoint(
        filepath=checkpoint_path,
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    )

    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=early_stop_patience,
        restore_best_weights=True,
        verbose=1
    )

    # -----------------------
    # PHASE 1: AGE ONLY
    # -----------------------

    dif_model_age_only.compile(
        optimizer=Adam(learning_rate=learning_rate, weight_decay=weight_decay),
        loss='mse',
        metrics=['mae']
    )

    dif_model_age_only.fit(
        X_train_pre, age_train,
        validation_data=(X_test_pre, age_test),
        epochs=phase1_epochs,
        batch_size=batch_size,
        callbacks=[reduce_lr, checkpoint],
        verbose=1
    )

    # -----------------------
    # PHASE 1.5: WARMUP ADV HEADS
    # -----------------------

    temp_adv_model = Model(
        inputs=input_layer,
        outputs=[gender_output, race_output]
    )

    temp_adv_model.compile(
        optimizer=Adam(learning_rate=learning_rate, weight_decay=weight_decay),
        loss={
            'gender_output': 'categorical_crossentropy',
            'race_output': 'categorical_crossentropy'
        }
    )

    temp_adv_model.fit(
        X_train_pre,
        {'gender_output': gen_train_cat,
         'race_output': race_train_cat},
        epochs=5,
        batch_size=batch_size,
        verbose=1
    )

    # -----------------------
    # PHASE 2: FULL ADVERSARIAL TRAINING
    # -----------------------

    mobilenet_base.trainable = True

    if n_unfreeze > 0:
        for layer in mobilenet_base.layers[:-n_unfreeze]:
            layer.trainable = False
        for layer in mobilenet_base.layers[-n_unfreeze:]:
            layer.trainable = True

    dif_model_full.compile(
        optimizer=Adam(learning_rate=learning_rate * 0.5, weight_decay=weight_decay),
        loss={
            'age_output': 'mse',
            'gender_output': 'categorical_crossentropy',
            'race_output': 'categorical_crossentropy'
        },
        loss_weights={
            'age_output': 1.0,
            'gender_output': adv_loss_weight,
            'race_output': adv_loss_weight
        },
        metrics={'age_output': 'mae'}
    )

    dif_model_full.fit(
        X_train_pre,
        {
            'age_output': age_train,
            'gender_output': gen_train_cat,
            'race_output': race_train_cat
        },
        validation_data=(
            X_test_pre,
            {
                'age_output': age_test,
                'gender_output': gen_test_cat,
                'race_output': race_test_cat
            }
        ),
        epochs=phase2_epochs,
        batch_size=batch_size,
        callbacks=[reduce_lr, checkpoint, early_stop],
        verbose=1
    )

    # -----------------------
    # EVALUATION
    # -----------------------

    if os.path.exists(checkpoint_path):
        dif_model_full.load_weights(checkpoint_path)

    preds = dif_model_full.predict(X_test_pre, batch_size=batch_size, verbose=0)
    pred_ages = np.array(preds[0]).flatten()

    overall_mae = mean_absolute_error(age_test.flatten(), pred_ages)

    gender_names = {0: 'male', 1: 'female'}
    race_names = {0: 'White', 1: 'Black', 2: 'Asian', 3: 'Indian', 4: 'Others'}

    gender_mae = group_mae(age_test, pred_ages, gen_test_labels, gender_names)
    race_mae = group_mae(age_test, pred_ages, race_test_arr, race_names)

    all_maes = list(gender_mae.values()) + list(race_mae.values())
    worst_group_mae = max(all_maes)
    mean_group_mae = np.mean(all_maes)
    bias_gap = max(all_maes) - min(all_maes)

    # -----------------------
    # LOG METRICS
    # -----------------------

    wandb.log({
        "overall_mae": overall_mae,
        "worst_group_mae": worst_group_mae,
        "mean_group_mae": mean_group_mae,
        "bias_gap": bias_gap
    })

    wandb.finish()

    return worst_group_mae

In [ ]:
# -----------------------
# EXECUTE W&B SWEEP
# -----------------------

sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'worst_group_mae',
        'goal': 'minimize'
    },
    'parameters': {
        'batch_size': {'values': [16, 32]},
        'learning_rate': {'values': [0.0001, 0.0005, 0.001]},
        'dropout_rate': {'values': [0.25, 0.3, 0.35]},
        'adv_loss_weight': {'values': [0.005, 0.01, 0.02]},
        'n_unfreeze': {'values': [25, 30, 35]},
        'reduce_lr_factor': {'values': [0.5]},
        'reduce_lr_patience': {'values': [3]},
        'early_stop_patience': {'values': [5]},
        'weight_decay': {'values': [0.0001]}
    }
}

sweep_id = wandb.sweep(
    sweep_config,
    project="age-gender-debiasing"
)

print("Sweep ID:", sweep_id)

wandb.agent(
    'ewgkcoz3',
    function=train_with_sweep,
    count=20
)

wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from /root/.netrc.


Create sweep with ID: 3javmb0y
Sweep URL: https://wandb.ai/lshearer2957-self/age-gender-debiasing/sweeps/3javmb0y
Sweep ID: 3javmb0y


wandb: Agent Starting Run: e3ttcqgq with config:
wandb: 	adv_loss_weight: 0.01
wandb: 	batch_size: 32
wandb: 	dropout_rate: 0.3
wandb: 	early_stop_patience: 5
wandb: 	learning_rate: 0.001
wandb: 	n_unfreeze: 35
wandb: 	reduce_lr_factor: 0.5
wandb: 	reduce_lr_patience: 3
wandb: 	weight_decay: 0.0001
wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from /root/.netrc.
wandb: Currently logged in as: lshearer2957 (lshearer2957-self) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin



Training with config:
{'adv_loss_weight': 0.01, 'batch_size': 32, 'dropout_rate': 0.3, 'early_stop_patience': 5, 'learning_rate': 0.001, 'n_unfreeze': 35, 'reduce_lr_factor': 0.5, 'reduce_lr_patience': 3, 'weight_decay': 0.0001}
Epoch 1/35
593/593 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 297.7363 - mae: 12.8495
Epoch 1: val_loss improved from inf to 159.11449, saving model to best_model_e3ttcqgq.keras
593/593 ━━━━━━━━━━━━━━━━━━━━ 51s 57ms/step - loss: 297.5901 - mae: 12.8463 - val_loss: 159.1145 - val_mae: 9.6965 - learning_rate: 0.0010
Epoch 2/35
590/593 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 160.6113 - mae: 9.6034
Epoch 2: val_loss improved from 159.11449 to 142.80302, saving model to best_model_e3ttcqgq.keras
593/593 ━━━━━━━━━━━━━━━━━━━━ 8s 13ms/step - loss: 160.6031 - mae: 9.6031 - val_loss: 142.8030 - val_mae: 8.8626 - learning_rate: 0.0010
Epoch 3/35
588/593 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 146.6706 - mae: 9.1416
Epoch 3: val_loss improved from 142.80302 to 138.22379

bias_gap,▁
mean_group_mae,▁
overall_mae,▁
worst_group_mae,▁
bias_gap,2.36751
mean_group_mae,6.07135
overall_mae,6.39304
worst_group_mae,7.192


...

wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 2335thn5 with config:
wandb: 	adv_loss_weight: 0.005
wandb: 	batch_size: 16
wandb: 	dropout_rate: 0.3
wandb: 	early_stop_patience: 5
wandb: 	learning_rate: 0.001
wandb: 	n_unfreeze: 35
wandb: 	reduce_lr_factor: 0.5
wandb: 	reduce_lr_patience: 3
wandb: 	weight_decay: 0.0001
wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from /root/.netrc.



Training with config:
{'adv_loss_weight': 0.005, 'batch_size': 16, 'dropout_rate': 0.3, 'early_stop_patience': 5, 'learning_rate': 0.001, 'n_unfreeze': 35, 'reduce_lr_factor': 0.5, 'reduce_lr_patience': 3, 'weight_decay': 0.0001}
Epoch 1/35
1186/1186 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 251.6448 - mae: 11.9625
Epoch 1: val_loss improved from inf to 144.94975, saving model to best_model_2335thn5.keras
1186/1186 ━━━━━━━━━━━━━━━━━━━━ 45s 25ms/step - loss: 251.5939 - mae: 11.9612 - val_loss: 144.9498 - val_mae: 9.0281 - learning_rate: 0.0010
Epoch 2/35
1179/1186 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 153.7699 - mae: 9.3598
Epoch 2: val_loss improved from 144.94975 to 141.01456, saving model to best_model_2335thn5.keras
1186/1186 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 153.7731 - mae: 9.3598 - val_loss: 141.0146 - val_mae: 8.8173 - learning_rate: 0.0010
Epoch 3/35
1181/1186 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 147.6999 - mae: 9.1900
Epoch 3: val_loss improved from 141.01456 to

bias_gap,▁
mean_group_mae,▁
overall_mae,▁
worst_group_mae,▁
bias_gap,3.00781
mean_group_mae,5.78475
overall_mae,6.17297
worst_group_mae,7.05039


wandb: Agent Starting Run: dlibyokv with config:
wandb: 	adv_loss_weight: 0.005
wandb: 	batch_size: 16
wandb: 	dropout_rate: 0.35
wandb: 	early_stop_patience: 5
wandb: 	learning_rate: 0.001
wandb: 	n_unfreeze: 35
wandb: 	reduce_lr_factor: 0.5
wandb: 	reduce_lr_patience: 3
wandb: 	weight_decay: 0.0001
wandb: [wandb.login()] Loaded credentials for https://api.wandb.ai from /root/.netrc.



Training with config:
{'adv_loss_weight': 0.005, 'batch_size': 16, 'dropout_rate': 0.35, 'early_stop_patience': 5, 'learning_rate': 0.001, 'n_unfreeze': 35, 'reduce_lr_factor': 0.5, 'reduce_lr_patience': 3, 'weight_decay': 0.0001}
